# Regression Notebook with PyGRN

In [1]:
from pygrn import grns, problems, evolution, config
import argparse
import os
import numpy as np
import matplotlib.pyplot as plt
from loguru import logger



# Creation of the Regression problem
def f(t, f: float = 1, k: int = 3):
    """ Fourrier decomposition of a square signal

    Args:
        t (float): time
        f(float, optional): frequency. Defaults to 1. 
        k (int, optional): degree of decomposition. Defaults to 2.

    Returns:
        values
    """
    y = np.zeros(t.shape[0])
    for i in range(0, k):

        y += np.sin((2*i + 1) * 2*np.pi*f*t)/(2*i + 1)
    
    
    y /= (4/np.pi)
    
    # transform values between 0 and 1
    y = (y - np.min(y)) / (np.max(y) - np.min(y))
    
    return y

def f_lin(t):

    return t

class Regression(problems.base.Problem):
    def __init__(self, x_train, y_train):
        super().__init__("regression")
        self.namestr = "regression"
        self.nin = 1
        self.nout = 1
        self.cacheable = True

        self.x_train = x_train
        self.y_train = y_train

        
    def eval(self, grn):

        grn.setup()
        grn.warmup(25)
        fit = 0.0
        for i in range(max(self.x_train.shape)):
            grn.set_input(self.x_train[i])
            grn.step()
            fit += np.linalg.norm(grn.get_output().item() - self.y_train[i]).item()

        return 1-fit


In [3]:
t = np.linspace(-1, 1, 500)
y = f(t, f=1, k=10)
    
# logger.remove()
logger.level("INFO")
grn = lambda : grns.ClassicGRN()
problem = MyRegression(t, y)


grneat = evolution.Evolution(problem, grn, num_workers=1)
best_fit, best_ind = grneat.run(100)

best_fit_history = grneat.best_fit_history
logger.info("best fit: ", best_fit)
# problem.eval(grneat.best_grn)
    
# y_eval = f(t_eval)
best_grn = best_ind.grn

best_grn.setup()
best_grn.warmup(25)
y_eval = []
for i in range(max(t.shape)):
    best_grn.set_input(t[i])
    best_grn.step()
    y_eval.append(best_grn.get_output().item())


plt.plot(t, y_eval * 2 - 1, label="prediction")
plt.plot(t, y, '.', label="target")

plt.legend()
plt.figure(figsize=(10, 5))
plt.plot(best_fit_history)
plt.show()

2025-07-17 16:19:55.045 | DEBUG    | pygrn.evolution.evolution:report:50 - S,2025-07-17T16:19:55.044812,0,0,58,0.482769,-206.187253,3,0.140000,0.054883,3.000000
2025-07-17 16:19:55.045 | DEBUG    | pygrn.evolution.evolution:report:50 - S,2025-07-17T16:19:55.045630,0,1,20,0.486182,-204.561554,3,0.160000,0.068396,3.000000
2025-07-17 16:19:55.046 | DEBUG    | pygrn.evolution.evolution:report:50 - S,2025-07-17T16:19:55.046000,0,2,10,0.482769,-206.187253,3,0.160000,0.027094,3.000000
2025-07-17 16:19:55.046 | DEBUG    | pygrn.evolution.evolution:report:50 - S,2025-07-17T16:19:55.046355,0,3,114,0.482786,-206.185713,3,0.140000,0.072179,3.000000
2025-07-17 16:19:55.047 | DEBUG    | pygrn.evolution.evolution:report:50 - S,2025-07-17T16:19:55.047205,0,4,48,0.482477,-206.167968,3,0.160000,0.059789,3.000000
2025-07-17 16:19:55.047 | INFO     | pygrn.evolution.evolution:report:78 - Generation 0: best fit -204.56155383540627, fit mean -206.1865451369865, fit std 0.14107628963351307
2025-07-17 16:19:5

TypeError: unsupported operand type(s) for -: 'list' and 'int'

In [ ]:
# test a random genome 
from pygrn import grns, problems, evolution, config
import argparse
import os
import numpy as np
import matplotlib.pyplot as plt
from loguru import logger



grn = grns.ClassicGRN()
grn.random(1, 1, 2)
t = np.linspace(0, 1, 100)

print("delta = ", grn.delta)
print("belta = ", grn.beta)
grn.setup()
grn.warmup(25)
y_eval = []
for i in range(max(t.shape)):
    grn.set_input(t[i])
    grn.step()

    print(grn.concentration)
    y_eval.append(grn.get_output().item())

plt.plot(t, y_eval, label="prediction")
# plt.plot(t, y, label="target")

plt.legend()
plt.figure(figsize=(10, 5))
# plt.plot(best_fit_history)
plt.show()

# Double the frequence of a sinus
- see [1] for explanantion 

In [ ]:
import numpy as np
import numba
from pygrn import grns, problems, evolution, config
import argparse
import os
import numpy as np
import matplotlib.pyplot as plt
from loguru import logger

class DoubleFreq(problems.base.Problem):


    def __init__(self, namestr="double_frequence", period = 10 ): 

        super().__init__(namestr)
        self.nin = 1
        self.nout = 1
        self.cacheable = True

        self.T = period

        t = np.linspace(-1, 1, 500)
        self.x_train = 0.5*np.sin(2*np.pi*t/self.T) + 0.5
        self.y_train = 0.5*np.sin(2*np.pi*t/(self.T/2)) + 0.5

    # numba.jit
    def eval(self, grn):
        grn.setup()
        grn.warmup(25)
        fit = 0.0
        for i in range(max(self.x_train.shape)):
            grn.set_input(self.x_train[i])
            grn.step()
            fit += np.linalg.norm(grn.get_output().item() - self.y_train[i]).item()


        return 1-fit


grn = lambda : grns.ClassicGRN()
problem = DoubleFreq()

grneat = evolution.Evolution(problem, grn, num_workers=10)

grneat.run(100)


best_fit_history = grneat.best_fit_history
logger.info("best fit: ", best_fit)
# problem.eval(grneat.best_grn)
    
# y_eval = f(t_eval)
best_grn = best_ind.grn

best_grn.setup()
best_grn.warmup(25)
y_eval = []
for i in range(max(t.shape)):
    best_grn.set_input(t[i])
    best_grn.step()
    y_eval.append(best_grn.get_output().item())


plt.plot(t, y_eval * 2 - 1, label="prediction")
plt.plot(t, y, '.', label="target")

plt.legend()
plt.figure(figsize=(10, 5))
plt.plot(best_fit_history)
plt.show()



2025-07-17 16:57:22.453 | DEBUG    | pygrn.evolution.evolution:report:50 - S,2025-07-17T16:57:22.453218,0,0,89,0.682635,-128.866875,3,0.140000,0.070555,3.000000
2025-07-17 16:57:22.454 | DEBUG    | pygrn.evolution.evolution:report:50 - S,2025-07-17T16:57:22.454237,0,1,112,0.669717,-136.497981,3,0.140000,0.070145,3.000000
2025-07-17 16:57:22.455 | DEBUG    | pygrn.evolution.evolution:report:50 - S,2025-07-17T16:57:22.455008,0,2,49,0.684264,-136.667636,3,0.160000,0.062116,3.000000
2025-07-17 16:57:22.455 | INFO     | pygrn.evolution.evolution:report:78 - Generation 0: best fit -128.8668746548456, fit mean -136.84297067806017, fit std 1.562196056879274
2025-07-17 16:57:23.631 | DEBUG    | pygrn.evolution.evolution:report:50 - S,2025-07-17T16:57:23.631679,1,0,22,0.000000,-249.000000,2,0.160000,0.080995,2.000000
2025-07-17 16:57:23.632 | DEBUG    | pygrn.evolution.evolution:report:50 - S,2025-07-17T16:57:23.632601,1,1,40,0.861962,-122.792985,3,0.140000,0.040603,3.550000
2025-07-17 16:57:23.

KeyboardInterrupt: 